In [3]:
import os
os.chdir('scratch/')

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
TRAIN_FILE = 'IRSE_FIRE_2022_Track_Training_Data_preprocessed.csv'
TEST_FILE = 'IRSE_FIRE_2022_Track_Test_Data_1000.csv'

In [ ]:
irse_df = pd.read_csv(TRAIN_FILE)

print(irse_df.info())

irse_df.head()

In [ ]:
test_df = pd.read_csv(TEST_FILE)

print(test_df.info())

test_df.head()

In [ ]:
classes = sorted(irse_df['Class'].value_counts().index.tolist())

classes

In [ ]:
irse_df['label'] = irse_df['Class'].apply(lambda l : classes.index(l))
test_df['label'] = test_df['Class'].apply(lambda l : classes.index(l))

irse_df.info(), test_df.info()

In [ ]:
irse_df = irse_df.drop_duplicates()

irse_df.info()

In [ ]:
train_df, dev_df = train_test_split(irse_df, test_size=0.1, 
    stratify=irse_df['label'])

train_df.shape, dev_df.shape

In [13]:
train_df.to_csv('irse.train.csv', index=False)
dev_df.to_csv('irse.dev.csv', index=False)

In [ ]:
pp_comments = irse_df['Comments']
pp_contexts = irse_df['Surrounding Code Context']
pp_labels = irse_df['label']
match_indexes = list()

for comment, context, label in zip(pp_comments, pp_contexts, pp_labels):
    match = test_df[(test_df['Comments'] == comment) & \
        (test_df['Surrounding Code Context'] == context)]
    match_indexes.extend(match.index.tolist())
    
    match_labels = match['label'].tolist()
    for ml in match_labels:
        assert ml == label
match_indexes = list(set(match_indexes))

len(match_indexes)

In [ ]:
test_df_pure = test_df.drop_duplicates().drop(index=match_indexes, errors='ignore')

print(test_df_pure.info())

test_df_pure.head()

In [13]:
test_df_pure.to_csv('irse.test.csv', index=False)

In [ ]:
test_df.to_csv('irse.given-test.csv', index=False)